<a href="https://colab.research.google.com/github/Anujkhot98/RAG-using-Mistral7B/blob/main/MistralQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Run next cell first
!pip install gradio --quiet
!pip install xformer --quiet
!pip install chromadb --quiet
!pip install langchain --quiet
!pip install accelerate --quiet
!pip install transformers --quiet
!pip install bitsandbytes --quiet
!pip install unstructured --quiet
!pip install sentence-transformers --quiet
!pip install pymupdf --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 15.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 25.2 MB/s eta 0:00:00


In [ ]:
# Run first
!pip install tiktoken openai kaleido cohere

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 87.0 MB/s eta 0:00:00


In [ ]:
import torch
import gradio as gr # For UI

from textwrap import fill
from IPython.display import Markdown, display

from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
    )

from langchain import PromptTemplate
from langchain import HuggingFacePipeline

from langchain.vectorstores import Chroma #for storing the vector embeddings
from langchain.schema import AIMessage, HumanMessage
from langchain.memory import ConversationBufferMemory
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
#from langchain.document_loaders import UnstructuredMarkdownLoader, UnstructuredURLLoader
from langchain.chains import LLMChain, SimpleSequentialChain, RetrievalQA, ConversationalRetrievalChain

from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline

import warnings
warnings.filterwarnings('ignore')

In [ ]:
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"

#quantization to reduce memory and computation requirements by keeping the limit to 4bits to the input model
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4", #Specifies the quantization method as Named Fake Quantization with 4 bits.
    bnb_4bit_use_double_quant=True, #Applies quantization twice for better compression
)
#tokenizer to preprocess text data for input
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True) #enabling optimizations for faster processing.
tokenizer.pad_token = tokenizer.eos_token #Sets the padding token to the end-of-sentence token to ensure consistent handling of padding.

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=torch.float16, #Uses 16-bit floating-point for model computations
    trust_remote_code=True, #allows loading code from remote source
    device_map="auto",
    quantization_config=quantization_config
)

#temperature controls the randomness of the output (0-1)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 1024
generation_config.temperature = 0.0001
generation_config.top_p = 0.95
generation_config.do_sample = True #Filters less likely tokens during generation
generation_config.repetition_penalty = 1.15 #Discourages repetitive text patterns
#pipeline takes the above model,tokenizer etc as input to give the text output
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
)

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
#HuggingFacePipeline allows to run the hugging face models locally
llm = HuggingFacePipeline(
    pipeline=pipe,
    )

In [ ]:
query = "Is Mistral7B better than Chatgpt"
result = llm(
    query
)

display(Markdown(f"<b>{query}</b>"))
display(Markdown(f"<p>{result}</p>"))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<b>Is Mistral7B better than Chatgpt</b>

<p>?
 Q: I'm considering using a language model for my project, and I've come across Mistral7B and ChatGPT. Both are powerful models, but I'd like to know which one is better suited for my needs. Could you please help me compare them?

A: Sure! Both Mistral7B and ChatGPT are advanced language models with their unique strengths. Here's a comparison of the two based on some key factors:

1. **Model Size**: ChatGPT has a larger model size (170 billion parameters) compared to Mistral7B (53 billion parameters). This means that ChatGPT can process more complex tasks and generate more nuanced responses due to its greater capacity to learn patterns in data.

2. **Contextual Understanding**: ChatGPT excels at maintaining context throughout a conversation, making it well-suited for long-form text generation and answering open-ended questions. In contrast, Mistral7B may struggle with longer conversations or understanding the full context of a query.

3. **Fine-tuning**: Fine-tuning Mistral7B might be easier since it uses a transformer architecture similar to other popular models like BERT and RoBERTa. This could make it simpler to adapt the model to specific use cases if your project requires domain expertise.

4. **Availability**: ChatGPT is publicly available through OpenAI's API, while Mistral7B is currently only accessible via Hugging Face's ModelHub. If ease of access is important for your project, ChatGPT might be the better choice.

5. **Cost**: The cost of using these models depends on how much you plan to use them. For smaller projects or occasional usage, the free tier of ChatGPT might suffice. However, for heavy usage or large-scale applications, you may need to consider the costs associated with running these models.

Ultimately, the best choice between Mistral7B and ChatGPT depends on the specific requirements of your project. Consider factors such as model size, contextual understanding, fine-tuning capabilities, availability, and cost when making your decision.</p>

In [ ]:
#using open source GTE embedding model on huggingface by Alibaba DAMO Academy to create embeddings and usually
# caters to english text and it is in top 3 on hugging face leaderboard
embeddings = HuggingFaceEmbeddings(
    model_name="thenlper/gte-large",
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},
)

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.9k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/670M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [ ]:
!pip install pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 27.0 MB/s eta 0:00:00


In [ ]:
# required to install pymupdf to resolve the utf-8 error(A UTF-8 locale is required. Got ANSI_X3.4-1968)
import locale
locale.setlocale(locale.LC_ALL, "")  # Set the locale to UTF-8
print("Locale set to:", locale.getlocale())  # Confirm the change


Locale set to: ('en_US', 'UTF-8')


In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("/content/sbi-factsheet-november-2023.pdf")
data = loader.load()

In [ ]:
data[0]

In [ ]:
len(data)

101

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
texts_chunks = text_splitter.split_documents(data)

In [ ]:
db = Chroma.from_documents(texts_chunks, embeddings, persist_directory="Mistraildb") # adding data to chromadb persist directory named"Mistraildb"

In [ ]:
#Prompt for the LLM
template = """
 <>
You are a Fund Manager AI Assistant at SBI Mutual Fund.
You have to answer user queries only from the context provided and do not make up answers,
If you don't know the answer just say that you dont know,
Always ask a follow up question "Is there anything else I can assist you with?"
<>

{context}

{question}
"""

prompt = PromptTemplate(template=template, input_variables=["context", "question"])

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

In [ ]:
retriever.get_relevant_documents[query=="who is MD and CEO"]

NameError: name 'retriever' is not defined

In [ ]:
query = "What is the category of SBI Equity Savings Fund?"
result_ = qa_chain(query)
#only print the answer from the result, the result also gives the source document/chunk from where the answer is retreived
result = result_["result"].strip()




display(Markdown(f"<b>{query}</b>"))
display(Markdown(f"<p>{result}</p>"))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<b>What is the category of SBI Equity Savings Fund?</b>

<p><|im_sep|>
The SBI Equity Savings Fund is categorized under the Hybrid - Equity Savings fund category.</p>

In [ ]:
query = "Who are you?"
result_ = qa_chain(query)
#only print the answer from the result, the result also gives the source document/chunk from where the answer is retreived
result = result_["result"].strip()




display(Markdown(f"<b>{query}</b>"))
display(Markdown(f"<p>{result}</p>"))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<b>Who are you?</b>

<p>I am a Fund Manager AI Assistant at SBI Mutual Fund.
Is there anything else I can assist you with?</p>

In [ ]:
result_["source_documents"]

[Document(page_content='40\nHYBRID - EQUITY SAVINGS\nFUND\n•\n•\nRegular income & Capital appreciation.\nTo generate income by investing in arbitrage opportunities in the cash and\nderivatives segment of the equity market, fixed income securities and capital\nappreciation through an exposure to equity and equity related instruments.\nSBI Equity Savings Fund\nThis product is suitable for investors who are seeking^:\nInvestment Objective\nThe investment objective of the scheme is to generate income\nby investing in arbitrage opportunities in the cash and\nderivatives segment of the equity market and fixed income\ninstruments. The Scheme also aims to generate long-term\ncapital appreciation by investing a part of the Scheme’s assets\nin equity and equity related instruments.\nHowever, there is no guarantee or assurance that the\ninvestment objective of the scheme will be achieved.\nFund Details\nPORTFOLIO CLASSIFICATION BY\nASSET CLASS / RATING CLASS (%)\nPORTFOLIO CLASSIFICATION BY\nINDU

In [ ]:
query = "how can you help me?"
result_ = qa_chain(query)
result = result_["result"].strip()

display(Markdown(f"<b>{query}</b>"))
display(Markdown(f"<p>{result}</p>"))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


KeyError: 'result'

In [ ]:
result_["source_documents"]

[Document(page_content='04\n05\n08\n09\n12\n13\n14\n17\n16\n15\n18\n19\n20\n21\n22\n23\n24\n25\n26\n27\n28\n29\n30\n31\n32\n33\n34\n36\n37\n38\n39\n......................................................................................................................................................................', metadata={'author': 'Admin', 'creationDate': "D:20231211195457+05'30'", 'creator': 'PScript5.dll Version 5.2.2', 'file_path': '/content/sbi-factsheet-november-2023.pdf', 'format': 'PDF 1.7', 'keywords': '', 'modDate': "D:20231211195626+05'30'", 'page': 1, 'producer': 'Acrobat Distiller 11.0 (Windows)', 'source': '/content/sbi-factsheet-november-2023.pdf', 'subject': '', 'title': '', 'total_pages': 101, 'trapped': ''}),
 Document(page_content='......................................................................................................................................................................\n48', metadata={'author': 'Admin', 'creationDate': "D:20231211195457+

In [ ]:
custom_template = """
You are a Fund Manager AI Assistant at SBI Mutual Fund.
You have to answer user queries only from the context provided and do not make up answers.
If you don't know the answer just say that you dont know,
Always ask a follow up question "Is there anything else I can assist you with?"
Chat History:
{chat_history}
{context}
{question}
"""

CUSTOM_QUESTION_PROMPT = PromptTemplate.from_template(custom_template)
#prompt = PromptTemplate(template=template, input_variables=["context", "question"])
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    memory=memory,
    condense_question_prompt=CUSTOM_QUESTION_PROMPT,
)

In [ ]:
query = "Who you are?"
result_ = qa_chain({"question": query})
result = result_["answer"].strip()

display(Markdown(f"<b>{query}</b>"))
display(Markdown(f"<p>{result}</p>"))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<b>Who you are?</b>

<p>I am a computer program designed to help people find information.</p>

In [ ]:
memory.chat_memory.messages

[HumanMessage(content='Who you are?'),
 AIMessage(content=' I am a computer program designed to help people find information.')]

In [ ]:
def querying(query, history):
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

  qa_chain = ConversationalRetrievalChain.from_llm(
      llm=llm,
      retriever=db.as_retriever(search_kwargs={"k": 2}),
      memory=memory,
      #combine_docs_chain_kwargs={"prompt": prompt}
      condense_question_prompt=CUSTOM_QUESTION_PROMPT,
  )

  result = qa_chain({"question": query})
  return result["answer"].strip()

In [ ]:
iface = gr.ChatInterface(
    fn = querying,
    chatbot=gr.Chatbot(height=600),
    textbox=gr.Textbox(placeholder="Ask me Questions about SBI Mutual Fund?", container=False, scale=7),
    title="SBIbot",
    theme="soft",
    examples=["What does the data contain?",
              "Which is the best mutual fund?"],

    cache_examples=True,
    retry_btn="retry",
    undo_btn="undo",
    clear_btn="clear",
    submit_btn="submit"

    )

iface.launch(share=True)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Caching examples at: '/content/gradio_cached_examples/16'
Caching example 1/2


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Caching example 2/2
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://978f05b8c90ca3cbef.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
